In [4]:
# Set up modules for Google functionality
from google.cloud import bigquery # To run BQ statements
from google_auth_oauthlib import flow # To authorise as user
from googleapiclient.discovery import build # To pull in from sheets, slides etc. API
from google.auth.transport.requests import Request
from google.cloud.bigquery import magics

import sys
sys.path.append(r'/home/jupyter/reusable_code')

import google_api_functions as gaf
creds=gaf.Authenticate_Google(r'/home/jupyter/reusable_code/') # Return logged-in credentials


In [5]:

bq = bigquery.Client(project='itv-bde-analytics-dev',credentials=creds) #Apply credentials to BQ client "bq"
magics.context.credentials = credentials  #apply these credentials to the BQ magic syntax too


In [10]:
for i in range(0,22):
    query="""

    insert `itv-bde-analytics-dev.britbox_analytics.weekly_engagement_funnel`  
    with
    entitled as 
    (select distinct britbox_id, timestamp_sub(timestamp_trunc(current_timestamp,WEEK(MONDAY)),INTERVAL (7*({}+1)) DAY) as week_beginning from `itv-bde-svod-prd.reporting.Entitlements_oldf_reporting` 
    where start_date<= 
    timestamp_sub(timestamp_sub(timestamp_trunc(current_timestamp,WEEK(MONDAY)),INTERVAL (7*{}) DAY),INTERVAL 1 MILLISECOND) --starts before the end of last week
    and end_date>=timestamp_sub(timestamp_trunc(current_timestamp,WEEK(MONDAY)),INTERVAL (7*({}+1)) DAY) --ends after the beginning of last week
    )

    ,logged_in as 
    (select distinct coalesce(TRIM(JSON_EXTRACT(raw,'$.user.itv_id'),"\\""),TRIM(JSON_EXTRACT(raw,'$.user.id'),"\\"")) as user_id, 1 as logged_in
    from `itv-bde-svod-prd.svod.svod_page_raw_in` 
    where timestamp_trunc(_PARTITIONTIME,WEEK(MONDAY))=timestamp_sub(timestamp_trunc(current_timestamp,WEEK(MONDAY)),INTERVAL (7*({}+1)) DAY) --last week
    )

    ,viewed as 
    (select itv_id, 1 as viewed, max(case when consumption/60000 >10 then 1 else 0 end) as viewed_10mins from `itv-bde-svod-prd.pes.svod_stream_summary`
    where timestamp_trunc(_PARTITIONTIME,WEEK(MONDAY))=timestamp_sub(timestamp_trunc(current_timestamp,WEEK(MONDAY)),INTERVAL (7*({}+1)) DAY) --last week
    group by 1
    )

    ,joined as (select
    a.*
    ,ifnull(b.logged_in,0) as logged_in
    ,ifnull(c.viewed,0) as viewed
    ,ifnull(c.viewed_10mins,0) as viewed_10mins

    from entitled a
    left join logged_in b
    on a.britbox_id=b.user_id

    left join viewed c
    on b.user_id=c.itv_id)

    select week_beginning,
    count(distinct britbox_id) as entitled_custs
    /*Absolutes*/
    ,count(*) as entitled
    ,sum(logged_in) as logged_in
    ,sum(viewed) as viewed
    ,sum(viewed_10mins) as viewed_10mins

    ,count(*)/count(*) as entitled_pc
    ,sum(logged_in)/count(*) as logged_in_pc
    ,sum(viewed)/count(*) as viewed_pc
    ,sum(viewed_10mins)/count(*) as viewed_10mins_pc


    ,sum(logged_in)/count(*) as conversion_to_login
    ,sum(viewed)/sum(logged_in) as conversion_to_view
    ,sum(viewed_10mins)/sum(viewed) as conversion_to_view_10mins

    from joined
    group by 1

    """.format(i,i,i,i,i)
    #print(query)
    bq.query(query)
    